# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Welcome to the world of DeepLearning.AI, a transformative platform revolutionizing the way we perceive artificial intelligence. Created by the renowned Andrew Ng, this cutting-edge program offers a gateway to the realms of deep learning, empowering enthusiasts worldwide. From convolutional neural networks to natural language processing, DeepLearning.AI equips you with the skills needed to thrive in this data-driven era. Whether you're a novice or an expert in AI, this platform caters to all, providing interactive courses and hands-on projects. Embark on this enlightening journey and unlock the endless possibilities of AI with DeepLearning.AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Welcome to the world of DeepLearning.AI, a transformative platform revolutionizing the way we perceive artificial intelligence. Created by the renowned Andrew Ng, this cutting-edge program offers a gateway to the realms of deep learning, empowering enthusiasts worldwide. From convolutional neural networks to natural language processing, DeepLearning.AI equips you with the skills needed to thrive in this data-driven era. Whether you're a novice or an expert in AI, this platform caters to all, providing interactive courses and hands-on projects. Embark on this enlightening journey and unlock the endless possibilities of AI with DeepLearning.AI.

-----------------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Welcome to the world of DeepLearning.AI, a transformative platform revolutionizing the way we perceive artificial intelligence. Created by the renowned Andrew Ng, this cutting-edge program offers a gateway to the realms of deep learning, empowering enthusiasts worldwide. From convolutional neural networks to natural language processing, DeepLearning.AI equips you with the skills needed to thrive in this data-driven era. Whether you're a novice or an expert in AI, this platform caters to all, providing interactive courses and hands-on projects. Embark on this enlightening journey and unlock the endless possibilities of AI with DeepLearning.AI.

-----------------------------------------

Critic (to Meta Reviewer):

Aggregrate feedback from all reviewers and give final suggestions on the writing.
Context: 
{'Reviewer': 'SEO Reviewer', 'Review': '- Consider using keywords related to "DeepLearning.AI" in the title, such as "DeepLearning.AI: Revolutionizing AI Education by Andrew Ng" to improve search engine visibility.\n- Include relevant terms like "deep learning courses" and "AI training" in the content to enhance organic search rankings and attract users looking for AI learning opportunities.\n- Add meta tags with keywords like "Andrew Ng," "DeepLearning.AI courses," and "artificial intelligence" to optimize the page further for search engines.'}
{'Reviewer': 'SEO Reviewer', 'Review': '- Consider using keywords related to "DeepLearning.AI" in the title, such as "DeepLearning.AI: Revolutionizing AI Education by Andrew Ng" to improve search engine visibility.\n- Include relevant terms like "deep learning courses" and "AI training" in the content to enhance organic search

## Get the summary

In [15]:
print(res.summary)

Title: DeepLearning.AI: Revolutionizing AI Education by Andrew Ng

Welcome to DeepLearning.AI, the ultimate hub for deep learning courses and AI training created by the renowned Andrew Ng. Dive into a world of convolutional neural networks and natural language processing, mastering the art of artificial intelligence. Whether you're a beginner or an AI expert, our interactive platform caters to all levels. Explore our curated courses, hands-on projects, and gain practical skills essential for the data-driven future. Join us on this transformative journey and unleash the power of AI with DeepLearning.AI. Embrace the future of technology today!
